# 2. 기본적인 추천시스템


## 2.1. 데이터 읽기

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import pandas as pd

base_src = "drive/MyDrive/RecoSys/python-recosys/Data"

In [4]:
# 사용자 u.user 파일 DataFrame으로 읽기

u_user_src = os.path.join(base_src, "u.user")
u_cols = ["user_id", "age", "sex", "occupation", "zipcode"]
users = pd.read_csv(u_user_src, sep = "|", names = u_cols, encoding = "latin-1")
users = users.set_index("user_id")
users.head()

,age,sex,occupation,zipcode
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [5]:
# 영화 u.item 파일 DataFrame으로 읽기

u_item_src = os.path.join(base_src, "u.item")
i_cols = ['movie_id','title','release date','video release date',
          'IMDB URL', 'unknown', 'Action', 'Adventure','Animation',
          'ChildrenI\'s','Comedy','Crime','Documentary','Drama','Fantasy',
          'Film-Noir', 'Horror', 'Musical' ,'Mystery','Romance','Sci-Fi','Thriller', 'War','Western']
movies = pd.read_csv(u_item_src, sep = "|", names = i_cols, encoding = "latin-1")
movies = movies.set_index("movie_id")
movies.head()

,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,ChildrenI's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
# 리뷰 u.data 파일 DataFrame으로 읽기

u_data_src = os.path.join(base_src, "u.data")
r_cols = ["user_id", "movie_id", "rating", "timestamp"]
ratings = pd.read_csv(u_data_src, sep = "\t", names = r_cols, encoding = "latin-1")
ratings = ratings.set_index("user_id")
ratings.head()

,movie_id,rating,timestamp
user_id,,,
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596


In [7]:
print(type(ratings))

<class 'pandas.core.frame.DataFrame'>


## 2.2 인기제품 방식
* 개별 사용자 정보 X
* 간단한 추천 제공
* 모든 사용자들에게 동일한 제품(Best-Seller) 추천  


In [8]:
def recom_movie(num_items) : #몇 개의 아이템 추천할 지
  #ratings에서 movie_id 기준으로 rating 컬럼에 대한 평균
  movie_mean = ratings.groupby(["movie_id"])["rating"].mean()

  #평균 냈던 거 정렬하는데, 특정 개수만큼만
  movie_sort = movie_mean.sort_values(ascending = False)[:num_items]

  #뽑힌 movie_sort의 index(movie_id)를 기준으로 movie dataFrame에서 조회
  #ratings에서는 영화 이름을 알 수 없으니까
  #loc() :특정 컬럼을 선택해 데이터 추출
  recom_movies = movies.loc[movie_sort.index]

  #추천된 영화의 title만 추출
  recommendations = recom_movies["title"]

  return recommendations

In [9]:
recom_movie(5)

movie_id
814                         Great Day in Harlem, A (1994)
1599                        Someone Else's America (1995)
1201           Marlene Dietrich: Shadow and Light (1996) 
1122                       They Made Me a Criminal (1939)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

## 2.3 추천시스템 정확도 측정
1. 훈련 데이터와 테스트 데이터로 나누기  
2. 정확도 측정하는 방법
  * RMSE 루트 민 스퀘어 에러   
    * n개의 데이터에 대해
    * (실제값-예측값) 제곱한 걸 더하고
    * n으로 나누고
    * 루트 씌우기  
    * 이 값이 **작아야** 좋은 것

In [10]:
#100K개의 영화 평점에 대해 실제값과 best-seller 방식으로 구한 예측값의 RMSE 계산하기

import numpy as np

def RMSE(y_true, y_pred) :
   return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))

# 정확도 계산
rmse = []
movie_mean = ratings.groupby(["movie_id"])["rating"].mean()

# 각 사용자가 평가한 모든 평점을 y_true에 저장
for user in set(ratings.index) :
  y_true = ratings.loc[user]["rating"]
  #best-seller 방식으로
  y_pred = movie_mean[ratings.loc[user]["movie_id"]]

  accuracy = RMSE(y_true, y_pred)
  rmse.append(accuracy)

# RMSE 계산
print(np.mean(rmse))

0.996007224010567


## 2.4 사용자 집단별 추천
* 훈련 데이터와 테스트 데이터 분리
* 성별 기준으로 집단 분리


In [11]:
# 데이터 불러오기
base_src = "drive/MyDrive/RecoSys/python-recosys/Data"

# 사용자
u_user_src = os.path.join(base_src, "u.user")
u_cols = ["user_id", "age", "sex", "occupation", "zipcode"]
users = pd.read_csv(u_user_src, sep = "|", names = u_cols, encoding = "latin-1")

# 영화
u_item_src = os.path.join(base_src, "u.item")
i_cols = ['movie_id','title','release date','video release date',
          'IMDB URL', 'unknown', 'Action', 'Adventure','Animation',
          'ChildrenI\'s','Comedy','Crime','Documentary','Drama','Fantasy',
          'Film-Noir', 'Horror', 'Musical' ,'Mystery','Romance','Sci-Fi','Thriller', 'War','Western']
movies = pd.read_csv(u_item_src, sep = "|", names = i_cols, encoding = "latin-1")

# 평점
u_data_src = os.path.join(base_src, "u.data")
r_cols = ["user_id", "movie_id", "rating", "timestamp"]
ratings = pd.read_csv(u_data_src, sep = "\t", names = r_cols, encoding = "latin-1")

In [12]:
# 데이터 전처리 추가

# 평점에서 timestamp 제거
ratings = ratings.drop("timestamp", axis = 1)
print(ratings.columns)

# 영화에서 movie_id와 title만 사용
movies = movies[["movie_id", "title"]]
print(movies.columns)

Index(['user_id', 'movie_id', 'rating'], dtype='object')
Index(['movie_id', 'title'], dtype='object')


In [13]:
# train set, test set 분리
from sklearn.model_selection import train_test_split

# 원본 데이터 보존 위해 복사
x = ratings.copy()
y = ratings["user_id"]

# stratify = y -> 개충하 추출 : 훈련과 테스트 데이터에서 y의 분포 비슷하게
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, stratify = y)


# 정확도(RMSE)계산하는 함수
def RMSE(y_true, y_pred) :
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))

# 모델별 RMSE 구하는 함수
def score(model) :
  # 테스트 데이터의 user_id와 movie_id의 쌍으로 맞추어 id_pairs 생성
  id_pairs = zip(x_test["user_id"], x_test["movie_id"])
  y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
  y_true = np.array(x_test["rating"])
  return RMSE(y_true, y_pred)

# best-seller 함수를 이용한 정확도 계산
train_mean = x_train.groupby(["movie_id"])["rating"].mean()

def best_seller(user_id, movie_id) :
  try :
    rating = train_mean[movie_id]

  # train set에는 없는데 test set에는 있는 영화
  except :
    rating = 3.0

  return rating

score(best_seller)
#정확도 지표로 사용하는 RMSE 값이 1.0245542042255897 정도로 나옴

1.0183668932462355

In [14]:
# 성별에 따른 예측값 계산

## x_train과 users를 user_id 기준으로 합치기 -> 조인
merged_ratings = pd.merge(x_train, users)

users = users.set_index("user_id")

## 성별에 따라 평균내기
g_mean = merged_ratings[["movie_id", "sex", "rating"]].groupby(["movie_id", "sex"])["rating"].mean()

## pandas의 pivot 메소드를 사용해 full matrix로 변환
### full matrix : 데이터의 모든 가능한 조합에 대한 값을 포함하는 행렬
### 예를 들면 사용자를 행으로, 아이템을 열로, 평점을 값으로 하는 풀 매트릭스로 변환
rating_matrix = x_train.pivot(index = "user_id", columns = "movie_id", values = "rating")



In [15]:
g_mean
# g_mean[movie_id]
# g_mean[movie_id].index

movie_id  sex
1         F      3.887755
          M      3.920635
2         F      3.083333
          M      3.185185
3         F      2.750000
                   ...   
1675      M      3.000000
1676      M      2.000000
1677      F      3.000000
1678      M      1.000000
1682      M      3.000000
Name: rating, Length: 3039, dtype: float64

In [16]:
# 성별 기준 추천
def cf_gender(user_id, movie_id) :
  if movie_id in rating_matrix.columns :
    # 해당 user_id의 성별
    gender = users.loc[user_id]["sex"]

    # 예측 대상인 영화가 해당 성별의 평균을 가지고 있는지 확인해야 함
    # 어떤 영화는 여자만 봤을 수도 있고, 아무도 평점을 남기지 않았을 수도 있는데
    # 아무도 평점을 남지기 않은 경우가 첫 번째 if문에서 확인한 거
    if gender in g_mean[movie_id].index :
      gender_rating = g_mean[movie_id][gender]
    else :
      gender_rating = 3.0

  # movie_id가 rating matrix에 없으면
  else :
    gender_rating = 3.0

  return gender_rating

score(cf_gender)
# 1.0385952297247518

1.0301347395073055

# 3. 협업 필터링 추천 시스템
* 실제 취향을 고려한 개인화 추천 시스템
* 이러한 아이디어로 만들어진 추천 알고리즘이 협업 필터링,   
  콜라보레이티브 필터링 CF
* 어떤 아이템에 대해 비슷한 취향을 가진 사람들은,  
  다른 아이템 또한 비슷한 취향을 가질 것이다

## 3.1. 협업 필터링의 원리
* 추천 대상과 취향이 비슷한 사람들의 집단이 먼저 만들어져 있다
* 취향이 비슷한 사람들을 찾아내기만 하면,  
  그 사람들이 좋아하는 아이템을 추천하면 된다  
* 유사도 계산 필요

## 3.2. 유사도 지표
* CF에서 사용자 간 유사도를 구하는 것이 핵심

### 3.2.1. 상관계수
* 가장 이해하기 쉬운 유사도
* -1과 +1 사이의 값  
  * -1이면 부적 상관 관계, 음의 상관 관계 (우하향?)
  * +1이면 양의 상관 관계 (우상향?)
  * 0이면 지표가 점으로 나타나 연속적이지 않음, 유사하지 않게 나타남, 유사도가 0이라는 뜻  
  * 그런데 대문자 U 모양으로 쿼드러틱하게 나타나는 경우도 유사도가 0으로 표현된다
  * 단순히 상관 관계가 0이라고 해서 유사도가 없다고 생각하면 안 되고,  
    데이터를 실제로 비주얼라이제이션 해보거나 해서 정확한 인사이트를 얻어야 한다
* 평가 자료가 연속형인 경우 가장 이해하기 쉬운 유사도

### 3.2.2. 코사인 유사도
* 협업 필터링에서 가장 널리 쓰이는 유사도
* 각 아이템 → 하나의 차원, 사용자의 평가값 → 좌표값
  * 예를 들면 x축은 아이템A, y축은 아이템B를 나타내고  
    점 C(x1, y1)에서 x1과 y1은 사용자C가 아이템A와 아이템B 각각에 대해 평가한 평가값을 나타낸다
* 두 사용자의 평가값 유사하다  
  → 코사인theta에서 theta는 작아지고 코사인 값은 커짐  
  (theta : 점A와 점B를 각각 원점과 이은 직선 사이의 각)
* -1과 +1 사이의 값
* 데이터가 이진값이라면(샀다 안 샀다/봤다 안 봤다) 타니모토 계수 사용 권장  

### 3.2.3. 자카드 계수
* 타니모토 계수의 변형
* 이진수 데이터라면 협업 필터링에서 좋은 결과 나타난다

## 3.3. 기본 CF 알고리즘